# Heterologous pathway implementation

In [8]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR

In [9]:
# Loading model
infilename = 'models/iMM904.xml'
print(f"Loading {infilename}")
model = read_sbml_model(infilename)

Loading models/iMM904.xml


In [10]:
# Model statistics
before_add = {"metabolites": len(model.metabolites), "reactions": len(model.reactions), "genes": len(model.genes)}
print("Metabolites:", before_add['metabolites']) #1226
print("Reactions:", before_add['reactions']) #1577
print("Genes:", before_add['genes']) #905

Metabolites: 1226
Reactions: 1577
Genes: 905


In [11]:
# Add all new metabolites from metabolites.csv
new_metabolites = dict()
with open("data/metabolites.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        m = Metabolite(
            line[0],
            formula=line[1],
            name=line[2],
            compartment=line[3])
        print(f'Adding metabolite {m.name} (id: {m})')
        new_metabolites[line[0]] = m

Adding metabolite cholesterol (id: cholesterol_c)
Adding metabolite 7-dehydrocholesterol (id: dehydrocholesterol_c)
Adding metabolite pregnenolone (id: pregnenolone_c)
Adding metabolite progesterone (id: progesterone_c)
Adding metabolite 5alpha-Cholesta-7_24-dien-3beta-ol (id: cholesta724dien3betaol_c)
Adding metabolite 5alpha-Cholest-8-en-3beta-ol (id: cholesta8en3betaol_c)
Adding metabolite lathosterol (id: lathosterol_c)
Adding metabolite 4-methylpentanal (id: methylpentanal_c)


In [12]:
# Add all new reactions from reactions.csv
with open("data/reactions.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        r = Reaction(line[0])
        r.name = line[2]
        r.subsystem = line[3]
        r.lower_bound = float(line[4])
        r.upper_bound = float(line[5])
        r.gpr = GPR.from_string(line[1])
        for i in range(int(len(line[6:])/2)):
            metaboliteID = line[6+i*2]
            if metaboliteID != "":
                if metaboliteID in model.metabolites:
                    metaboliteID = model.metabolites.get_by_id(metaboliteID)
                else:
                    metaboliteID = new_metabolites[metaboliteID]
                bin = float(line[7+i*2])
                r.add_metabolites({metaboliteID:bin})
        print(f'Adding reaction {r} | enzyme {r.gpr}')
        model.add_reactions([r])

        

Adding reaction R07498: h_c + nadph_c + zymst_c <=> cholesta8en3betaol_c + nadp_c | enzyme DHCR24
Adding reaction R05703: cholesta724dien3betaol_c + h_c + nadph_c <=> lathosterol_c + nadp_c | enzyme DHCR24
Adding reaction R01456: dehydrocholesterol_c + h_c + nadph_c --> cholesterol_c + nadp_c | enzyme DHCR7
Adding reaction ECYP11A1: cholesterol_c + 6.0 h_c + 6.0 nadph_c + 3.0 o2_c --> 4.0 h2o_c + methylpentanal_c + 6.0 nadp_c + pregnenolone_c | enzyme CYP11A1
Adding reaction R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c | enzyme HSD3B
Adding reaction R03353: cholesta8en3betaol_c --> lathosterol_c | enzyme YMR202W
Adding reaction R07215: h_c + lathosterol_c + nadph_c + o2_c --> dehydrocholesterol_c + 2.0 h2o_c + nadp_c | enzyme YLR056W
Adding reaction R04804: zymst_c --> cholesta724dien3betaol_c | enzyme YMR202W


In [13]:
# New model statistics
print("Metabolites:", len(model.metabolites), f"(change: {len(model.metabolites) - before_add['metabolites']})") 
print("Reactions:", len(model.reactions), f"(change: {len(model.reactions) - before_add['reactions']})") 
print("Genes:",len(model.genes), f"(change: {len(model.genes) - before_add['genes']})") 

Metabolites: 1234 (change: 8)
Reactions: 1585 (change: 8)
Genes: 909 (change: 4)


In [14]:
# Saving new model
outfilename = "models/iMM904_progesterone.xml"
print(f"Saving to {outfilename}")
write_sbml_model(model, outfilename)

Saving to models/iMM904_progesterone.xml
